In [ ]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from Levenshtein import distance
warnings.simplefilter("ignore")
%matplotlib notebook

In [ ]:
bano = pd.read_csv("https://bano.openstreetmap.fr/data/bano-78.csv", names=["id", "numero", "voie", "code_post", "nom_comm", "source", "lat", "lon"])

In [ ]:
bano

In [ ]:
bano.describe()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_aspect('equal')
plot = ax1.scatter(bano.lon, bano.lat, s=.1)

In [ ]:
bano.source

In [ ]:
bano.source = pd.Categorical(bano.source)
fig1, ax1 = plt.subplots()
ax1.set_aspect('equal')
plot = ax1.scatter(bano.lon, bano.lat, s=1, c=bano.source.cat.codes)

In [ ]:
bano.voie.str.split().str[0].value_counts().head(15).index

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_aspect('equal')
for kind in bano.voie.str.split().str[0].value_counts().head(15).index:
    to_plot = bano[bano.voie.str.startswith(kind)]
    ax1.scatter(to_plot.lon, to_plot.lat, s=.1, label=kind)
ax1.legend(markerscale=10, fontsize='small', bbox_to_anchor=(1.05, .95),  loc='upper left', )

In [ ]:
bano.numero.max()

In [ ]:
pd.to_numeric(bano.numero, errors='coerce').max()

In [ ]:
bano["numero_as_number"] = pd.to_numeric(bano["numero"], errors='coerce')

In [ ]:
bano[bano.numero == "78250"]

In [ ]:
bano[(bano.numero_as_number > 3000) & (bano.numero_as_number < 5000)]

In [ ]:
bano[~bano.code_post.astype(str).str.startswith("78")]

In [ ]:
road_name_counts = bano.voie.value_counts()

In [ ]:
road_name_counts

In [ ]:
road_name_counts[road_name_counts == 1]

In [ ]:
epsilon = 0.01
for suspicious_name in road_name_counts[road_name_counts == 1].index:
    lat, lon = bano[bano.voie == suspicious_name].loc[:,('lat', 'lon')].to_numpy()[0]
    neighbors = bano[(lat - epsilon < bano.lat) & (bano.lat < lat + epsilon) & (lon - epsilon < bano.lon) & (bano.lon < lon + epsilon)]
    neighbors_names = set(neighbors.voie.values) - set([suspicious_name])
    if not neighbors_names:
        continue
    proposed_name = min(neighbors_names, key=lambda n: distance(n, suspicious_name))
    if distance(proposed_name, suspicious_name) < 3:
        print(f"Proposing to replace {suspicious_name!r} by {proposed_name!r}")
    

In [ ]:
nom_comm_counts = bano.nom_comm.value_counts()
print(nom_comm_counts)